In [ ]:
# #GFS data takes a while to download from NCAR servers. 
# #Uncomment to download unprocessed GFS data from Google Drive
# !gdown -O pm-gfs-interim.zip 1mzK2aU922huX9FRm7OJz-EQr0OfXZioQ
# !unzip -qqn pm-gfs-interim.zip

In [ ]:
# #Uncomment to download preprocessed data
# !gdown -O pm-data.zip 1-EgerNIBSDM5gjQvs2j5B53CfW9MvAzW
# !unzip -qqn pm-data.zip

In [2]:
#Uncomment to download prepretrained models
# !gdown -O pm-models.zip 1-Ia68IOdGNHvaWoLI6yfSF9NxuueLym8
# !mkdir pm-models
!unzip -qqn pm-models.zip -d pm-models/


In [ ]:
##Extract NASADEM elevation data
!python3 src/data/extract_nasadem.py --data_dir data

In [2]:
##Extract MAIAC data from aws
!python3 src/data/extract_maiac.py --data_dir data --path_satmeta data/raw/pm25_satellite_metadata.csv --stage test --s3url us_url --cleanup --n_jobs 32 


using 32 processes
QUEUEING TASKS | : 100%|██████████████████| 2444/2444 [00:00<00:00, 5883.63it/s]
PROCESSING TASKS | : 100%|██████████████████| 2444/2444 [40:52<00:00,  1.00s/it]
COLLECTING RESULTS | : 100%|████████████| 2444/2444 [00:00<00:00, 528613.81it/s]
Saved data to data/interim/maiac/maiac_test.csv


In [ ]:
##Extract GFS data
!python3 src/data/extract_gfs.py --data_dir data --stage test --labels data/raw/submission_format.csv --n_jobs 16 

In [8]:
##Create final datasets
!python3 src/data/create_dataset.py --data_dir data --stage test --labels data/raw/submission_format.csv --n_gfs 8 --n_jobs 4
!python3 src/data/create_dataset.py --data_dir data --stage test --labels data/raw/submission_format.csv --n_gfs 10 --n_jobs 4
!python3 src/data/create_dataset.py --data_dir data --stage test --labels data/raw/submission_format.csv --n_gfs 12 --n_jobs 4


QUEUEING TASKS | : 100%|███████████████| 13504/13504 [00:00<00:00, 22959.83it/s]
PROCESSING TASKS | : 100%|████████████████| 13504/13504 [03:26<00:00, 65.54it/s]
COLLECTING RESULTS | : 100%|██████████| 13504/13504 [00:00<00:00, 604223.18it/s]
sys:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
Saved data to data/processed/test/test_tail8.pkl
QUEUEING TASKS | : 100%|███████████████| 13504/13504 [00:00<00:00, 24217.27it/s]
PROCESSING TASKS | : 100%|████████████████| 13504/13504 [03:27<00:00, 64.97it/s]
COLLECTING RESULTS | : 100%|██████████| 13504/13504 [00:00<00:00, 618481.11it/s]
sys:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

In [3]:
##Generate predictions - Writes test set predictions to Submission.csv
!python3 src/inference.py --data_dir data --model_dir pm-models/models --stage test --subformat_path data/raw/submission_format.csv --output_path Submission_oldmodel_newdata.csv 


##### RUNNING INFERENCE FOR DATASET 8 ##########
location 2, fold: 0, samples:2507 4.347227857605272,30.86453188614086
location 2, fold: 1, samples:2507 7.517943322969136,28.891477288786376
location 2, fold: 2, samples:2507 5.563945362223738,30.216576848991924
location 2, fold: 3, samples:2507 7.176982673480211,31.241455802315063
location 2, fold: 4, samples:2507 9.028134998845683,28.15961723037245
location 1, fold: 0, samples:6333 0.6344122219428749,60.12124442318062
location 1, fold: 1, samples:6333 2.57244551470458,32.57510437953324
location 1, fold: 2, samples:6333 2.6139003814105504,28.197874765411164
location 1, fold: 3, samples:6333 2.6872031831692285,33.98659733505045
location 1, fold: 4, samples:6333 0.29114658103774854,29.907534914309398
location 0, fold: 0, samples:4664 17.002346351221263,388.39588290038506
location 0, fold: 1, samples:4664 17.599256852313452,416.4992376648482
location 0, fold: 2, samples:4664 12.002972630233197,404.6589271241406
location 0, fold: 3, samples

In [ ]:
# import pandas as pd
# sub=pd.read_csv('Submission.csv')
# sub